# **Setup**

In [1]:
# download data
# !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1nckRRVYRiWG8VWoUY4wwtuQm6DjIbpmo' -O w_review_train.csv

# # download font for matplot
# !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1DsfYE5uI1ZA_IXDNkezQTv-NQehxgZQy' -O THSarabun.ttf

# install library
!pip install pythainlp
# get tensorflow
# !pip install --upgrade pip
!pip install tensorflow
!pip install scikit-learn

# Import Library
---

In [2]:
from tensorflow import keras

import pandas as pd
import numpy as np
from pythainlp import word_tokenize
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import confusion_matrix, classification_report

---
### Prepare Data {Tokenize Word}
---

In [3]:
# check data
data = pd.read_csv("NLP_Elder_Companion.csv")
print(data.shape)
print(data.head())

# word tokenize
comment = data["Text"].apply(word_tokenize, engine="newmm", keep_whitespace=False)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(comment)
comment = tokenizer.texts_to_sequences(comment)

# adjust sequence to have same size
maxlen = max([len(s) for s in comment])
print("max len", maxlen)
x = pad_sequences(comment, maxlen=maxlen, padding="post")

y = np.array(data["Class"], dtype=np.int32).reshape(-1, 1)

# One-hot encode the target data
num_classes = 10
y = to_categorical(y, num_classes=num_classes)

xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2, random_state=22)

(477, 2)
   Class                        Text
0      0         คนแก่กินอะไรได้บ้าง
1      0                ควรกินอะไรดี
2      0               อาหารที่แนะนำ
3      0              ทานอะไรได้บ้าง
4      0  อาหารที่เหมาะกับผู้สูงอายุ
max len 22


In [4]:
from keras.layers import SimpleRNN, LSTM, Embedding, Flatten, Dense, Bidirectional
from keras.models import Sequential, Model
from keras.losses import CategoricalCrossentropy
from keras.optimizers import Adam

In [6]:
# Tokenizer().word_index = word index
# +1 is from function paramiter
vocabSize = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(input_dim=vocabSize,
                   output_dim=2000,
                   input_length=maxlen))
model.add(Bidirectional(LSTM(2000)))
model.add(Dense(10, activation="softmax"))

model.summary()

lost_func = CategoricalCrossentropy()
otm = Adam()
model.compile(optimizer=otm, loss=lost_func, metrics=["accuracy"])

model.fit(xTrain, yTrain, batch_size=46, epochs=2, verbose=1)

y_predict = model.predict(xTest)
print(model.evaluate(xTest, yTest, verbose=1))

y_predict = np.argmax(y_predict, axis=1)
cm = confusion_matrix(np.argmax(yTest, axis=1), y_predict)
report = classification_report(np.argmax(yTest, axis=1), y_predict)
print("Confusion Matrix:")
print(cm)

print("\nClassification Report:")
print(report)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 22, 2000)          1190000   
                                                                 
 bidirectional_1 (Bidirecti  (None, 4000)              64016000  
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, 10)                40010     
                                                                 
Total params: 65246010 (248.89 MB)
Trainable params: 65246010 (248.89 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/2
9/9 [==============================] - 56s 6s/step - loss: 2.3141 - accuracy: 0.1680
Epoch 2/2
3/3 [==============================] - 2s 490ms/step - loss: 1.3627 - accuracy: 0.5729
[1.3626703023910

C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Lenovo\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Tokenizer().word_index = word index
# +1 is from function paramiter
vocabSize = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(input_dim=vocabSize,
                   output_dim=2000,
                   input_length=maxlen))
model.add(LSTM(2000))
model.add(Dense(10, activation="softmax"))

model.summary()

lost_func = CategoricalCrossentropy()
otm = Adam()
model.compile(optimizer=otm, loss=lost_func, metrics=["accuracy"])

print("x", xTrain.shape, ":", "y", yTrain.shape)
model.fit(xTrain, yTrain, batch_size=46, epochs=3, verbose=1)

y_predict = model.predict(xTest)
print(model.evaluate(xTest, yTest, verbose=1))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 22, 2000)          1190000   
                                                                 
 lstm_2 (LSTM)               (None, 2000)              32008000  
                                                                 
 dense_2 (Dense)             (None, 10)                20010     
                                                                 
Total params: 33218010 (126.72 MB)
Trainable params: 33218010 (126.72 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
x (381, 22) : y (381, 10)
Epoch 1/3
9/9 [==============================] - 28s 3s/step - loss: 2.4560 - accuracy: 0.0971
Epoch 2/3
9/9 [==============================] - 26s 3s/step - loss: 2.3230 - accuracy: 0.1181
Epoch 3/3
5/9 [===============>..............] - ETA: 11s 

In [ ]:
# Tokenizer().word_index = word index
# +1 is from function paramiter
vocabSize = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(input_dim=vocabSize,
                   output_dim=2000,
                   input_length=maxlen))
model.add(SimpleRNN(2000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(500,  activation="relu"))
model.add(Dense(10, activation="softmax"))

model.summary()

lost_func = CategoricalCrossentropy()
otm = Adam()
model.compile(optimizer=otm, loss=lost_func, metrics=["accuracy"])

print("x", xTrain.shape, ":", "y", yTrain.shape)
model.fit(xTrain, yTrain, batch_size=46, epochs=3, verbose=1)

y_predict = model.predict(xTest)
print(model.evaluate(xTest, yTest, verbose=1))